# NameX Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [20]:
import os
import psycopg2
import sqlalchemy
import pandas as pd
import csv
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


This will create the connection to the database and prep the jupyter magic for SQL...

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('DATABASE_USERNAME', '') + ":" + os.getenv('DATABASE_PASSWORD', '') +'@' + \
                os.getenv('DATABASE_HOST', '') + ':' + os.getenv('DATABASE_PORT', '') + '/' + os.getenv('DATABASE_NAME', '');

%sql $connect_to_db


Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

Daily totals for specified date: Following query, 'current_date - 0' means today, 'current_date - 1' means yesterday, 'current_date - 2' means the day before yesterday...
# date((r.last_update AT TIME ZONE 'UTC') AT TIME ZONE 'PST') = date(now() AT TIME ZONE 'PST')  ???
# date(r.last_update AT TIME ZONE 'PST') = current_date ???

In [ ]:
%%sql stat_daily_completed  <<
SELECT to_char(date(e.event_dt), 'YY-Mon-DD') AS Examined_Date
     , (select username from users u where u.id=e.user_id) AS EXAMINER     
     , count(e.*) + count(r.*) FILTER (WHERE r.priority_cd = 'Y')   AS total                  
FROM events e,
requests r
WHERE e.user_id != 1
AND r.id = e.nr_id
AND date( e.event_dt) = date(current_date -1)
AND e.state_cd in ('APPROVED','REJECTED','CONDITIONAL','CANCELLED')
group by e.user_id, date(e.event_dt);

In [ ]:
edt = stat_daily_completed.DataFrame()

if not edt.empty: 
    edt['examiner'] = edt['examiner'].str.replace('idir/','')        
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(HTML(edt.to_html()))
        print('grand total', edt['total'].sum())
        
    

Save to CSV
    

In [ ]:
filename = 'daily_totals_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'
edt.to_csv(filename, sep=',', encoding='utf-8', index=False)

if edt.empty:
    with open(filename, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(('No Data Retrieved',''))